#### get scene ids for bbox area and bein and end datetimes

In [1]:
from lsru import Usgs
import datetime
import pandas as pd

from lsru import Espa
from pprint import pprint

# Instantiate Espa class
espa = Espa()

# Instantiate Usgs class and login
usgs = Usgs()
usgs.login()

True

In [2]:
# Define query extent
bbox = (-120.66, 34.42, -120.30, 34.68) # bb for dangermond and sb. left bottom right top
all_scene_ids = []
for l_num in [5, 7, 8]:
    coll_name = Usgs.get_collection_name(l_num)
    # Query the Usgs api to find scene intersecting with the spatio-temporal window
    scene_list = usgs.search(collection=coll_name,
                             bbox=bbox,
                             begin=datetime.datetime(1985,1,1),
                             end=datetime.datetime(2020,11,3),
                             max_results=100000,
                             max_cloud_cover=5)

    # Extract Landsat scene ids for each hit from the metadata
    scene_ids = [x['displayId'] for x in scene_list]
    all_scene_ids.extend(scene_ids)

#### Subset by tier (quality) and daytime by using the path/row (needs to change depending on the bbox)

In [11]:
df = pd.DataFrame({"scene_id":all_scene_ids}).reindex()
split_vals_series = df.scene_id.str.split("_")

# https://www.usgs.gov/faqs/how-can-i-tell-difference-between-landsat-collections-data-and-landsat-data-i-have-downloaded?qt-news_science_products=0#qt-news_science_products
df = pd.DataFrame(split_vals_series.to_list(), columns=['sensor_satellite', 'correction_level', 'path_row', 'acquisition_date', 'espa_date', 'collection_number', 'tier'])
df['scene_id'] = all_scene_ids

In [12]:
df['path_row'].unique()

array(['043036', '042036', '141208', '140208'], dtype=object)

In [13]:
df = df[df['path_row'].str.contains("042036")] # daytime pathrows
print(len(df))

df = df[df['tier']=='T1'] # highest quality data with <12 RMSE terrain correction

print(len(df))

270
267


#### Scene availability plots

In [14]:
df['datetime'] = pd.to_datetime(df['acquisition_date'])
df['year'] = df.datetime.apply(lambda x: x.year)
df['month'] = df.datetime.apply(lambda x: x.month)

In [15]:
import altair as alt
alt.Chart(df).mark_bar().encode(
    x='year',
    color='sensor_satellite',
    y='count()'
).interactive()

alt.Chart(...)

In [16]:
alt.Chart(df).mark_bar().encode(
    x='month',
    color='sensor_satellite',
    y='count()'
).interactive()

alt.Chart(...)

In [19]:
espa.get_available_products(df['scene_id'].to_list())

{'tm5_collection': {'products': ['source_metadata',
   'l1',
   'toa',
   'bt',
   'sr',
   'sr_ndvi',
   'sr_evi',
   'sr_savi',
   'sr_msavi',
   'sr_ndmi',
   'sr_nbr',
   'sr_nbr2',
   'stats',
   'pixel_qa',
   'et'],
  'inputs': ['LT05_L1TP_042036_19850131_20161004_01_T1',
   'LT05_L1TP_042036_19850320_20161004_01_T1',
   'LT05_L1TP_042036_19850912_20161004_01_T1',
   'LT05_L1TP_042036_19851014_20161004_01_T1',
   'LT05_L1TP_042036_19860118_20161004_01_T1',
   'LT05_L1TP_042036_19860203_20161004_01_T1',
   'LT05_L1TP_042036_19860915_20161003_01_T1',
   'LT05_L1TP_042036_19861102_20161003_01_T1',
   'LT05_L1TP_042036_19870121_20161003_01_T1',
   'LT05_L1TP_042036_19870206_20161003_01_T1',
   'LT05_L1TP_042036_19870326_20161003_01_T1',
   'LT05_L1TP_042036_19880328_20161003_01_T1',
   'LT05_L1TP_042036_19880429_20161003_01_T1',
   'LT05_L1TP_042036_19881209_20161002_01_T1',
   'LT05_L1TP_042036_19890126_20161002_01_T1',
   'LT05_L1TP_042036_19890227_20161002_01_T1',
   'LT05_L1TP_0

### data only available in CONUS

In [20]:
from lsru import Espa
from pprint import pprint

# Instantiate Espa class
espa = Espa()

# Place order (full scenes, no reprojection, sr and pixel_qa)
order = espa.order(scene_list=df['scene_id'].to_list(), products=['et', 'sr', 'pixel_qa', 'source_metadata', 'stats'])
print(order.orderid)

espa-ravery@ucsb.edu-11052020-141757-537


## Start here if continuing from existing order that needs to be monitored and downloaded

In [24]:
from lsru import Espa
from pprint import pprint

# Instantiate Espa class
espa = Espa()


for order in espa.orders:
    # Orders have their own class with attributes and methods
    print('%s: %s' % (order.orderid, order.status))

for order in espa.orders:
    if order.is_complete:
        order.download_all_complete('/home/rave/serdp/dangermond-sb/landsat-sr-et/')

### Needs to be inspected and adjusted for each particular area

In [ ]:
pprint(espa.projections['utm'])

In [ ]:
# Inspect aea projection parameters
pprint(espa.projections['aea'])
# Define projection parameters
proj_params = {'aea': {'central_meridian': 3.8,
                       'datum': 'wgs84',
                       'false_easting': 0,
                       'false_northing': 0,
                       'latitude_of_origin': 43.7,
                       'standard_parallel_1': 43,
                       'standard_parallel_2': 44}}
# Place order
order_meta = espa.order(scene_list=scene_list, products=['et', 'sr', 'pixel_qa', 'sr_nbr2', 'sr_evi', 'source_metadata', 'stats'],
                        note='cropped order with resampling', projection=proj_params,
                        extent=bbox, resolution=60)

#### below is the lower level api, lsru is an easier package to work with, but this might be needed later.

In [ ]:
import platform
print(platform.python_version())
import requests
import json
import getpass
host = 'https://espa.cr.usgs.gov/api/v1/'



username = 'rbavery'
password = getpass.getpass()

def espa_api(endpoint, verb='get', body=None, uauth=None):
    """ Suggested simple way to interact with the ESPA JSON REST API """
    auth_tup = uauth if uauth else (username, password)
    response = getattr(requests, verb)(host + endpoint, auth=auth_tup, json=body)
    print('{} {}'.format(response.status_code, response.reason))
    data = response.json()
    if isinstance(data, dict):
        messages = data.pop("messages", None)  
        if messages:
            print(json.dumps(messages, indent=4))
    try:
        response.raise_for_status()
    except Exception as e:
        print(e)
        return None
    else:
        return data

In [ ]:
print('GET /api/v1/user')
resp = espa_api('user')
print(json.dumps(resp, indent=4))

In [ ]:
print('GET /api/v1/available-products')
avail_list = {'inputs': ['LC08_L1TP_197030_20130416_20170505_01_T1'
                        ]
             }
resp = espa_api('available-products', body=avail_list)
print(json.dumps(resp, indent=4))

In [ ]:
print('GET /api/v1/projections')
projs = espa_api('projections')
print(projs.keys())

In [ ]:
print(json.dumps(projs['utm']['properties'], indent=4))

In [ ]:
l8_ls = ['LC08_L1TP_029030_20161109_20170219_01_T1',
         'LC08_L1TP_029030_20160821_20170222_01_T1',
         'LC08_L1TP_029030_20130712_20170309_01_T1']
l7_ls =['LE07_L1TP_029030_20170221_20170319_01_T1',
        'LE07_L1TP_029030_20161101_20161127_01_T1',
        'LE07_L1TP_029030_20130602_20160908_01_T1']

# Differing products across the sensors
l7_prods = ['toa', 'bt']
l8_prods = ['sr']

# Standard Albers CONUS
projection = {'aea': {'standard_parallel_1': 29.5,
                      'standard_parallel_2': 45.5,
                      'central_meridian': -96.0,
                      'latitude_of_origin': 23.0,
                      'false_easting': 0,
                      'false_northing': 0,
                      'datum': 'nad83'}}

# Let available-products place the acquisitions under their respective sensors
ls = l8_ls + l7_ls

print('GET /api/v1/available-products')
order = espa_api('available-products', body=dict(inputs=ls))
print(json.dumps(order, indent=4))

In [ ]:
# Replace the available products that was returned with what we want
for sensor in order.keys():
    if isinstance(order[sensor], dict) and order[sensor].get('inputs'):
        if set(l7_ls) & set(order[sensor]['inputs']):
            order[sensor]['products'] = l7_prods
        if set(l8_ls) & set(order[sensor]['inputs']):
            order[sensor]['products'] = l8_prods

# Add in the rest of the order information
order['projection'] = projection
order['format'] = 'gtiff'
order['resampling_method'] = 'cc'
order['note'] = 'API Demo Jupyter!!'

# Notice how it has changed from the original call available-products
print(json.dumps(order, indent=4))

In [ ]:
# Place the order
print('POST /api/v1/order')
resp = espa_api('order', verb='post', body=order)
print(json.dumps(resp, indent=4))

In [ ]:
resp['orderid']

In [ ]:
print('GET /api/v1/order-status/{}'.format(orderid))
resp = espa_api('order-status/{}'.format(orderid))
print(json.dumps(resp, indent=4))

In [ ]:
print('GET /api/v1/item-status/{0}'.format(orderid))
resp = espa_api('item-status/{0}'.format(orderid), body={'status': 'complete'})
print(json.dumps(resp[orderid], indent=4))

In [ ]:
# Once the order is completed or partially completed, can get the download url's
for item in resp[orderid]:
    print("URL: {0}".format(item.get('product_dload_url')))